In [2]:
import numpy as np 
import math 

L=4 
d_k=8 
d_v=8 

q =  np.random.randn(L,d_k)
k = np.random.randn(L,d_k)
v = np.random.randn(L,d_v)

In [3]:
print("Q\n",q )
print("K\n",k )
print("V\n",v )


Q
 [[-1.43357575 -0.43065121  0.77136426  0.19574309 -0.22401895 -1.04299148
   1.10825192  0.20315761]
 [-0.34115764 -1.5118362   0.01821113 -1.5611847  -0.76708133  2.09346769
  -0.67368897 -1.39134768]
 [ 0.53909654  0.6232494   1.74257407  1.2956597   0.18971572 -1.70160811
  -1.06915727 -1.02722077]
 [-0.74246299 -1.87555485  0.99599366  0.55042828 -0.56317419 -0.71420677
  -1.07514025 -1.38289993]]
K
 [[-1.36513690e+00 -3.63309409e-01 -5.99209478e-01 -9.82497463e-01
  -1.68603158e+00  4.11995122e-02  1.12483738e-01 -7.87179268e-02]
 [-1.16276551e+00  1.76990511e-01 -4.58214292e-01  1.15299911e-01
   6.41515596e-01 -7.75446731e-01 -1.57954247e+00  2.88702225e-03]
 [-1.02698577e-01 -9.33278768e-01 -1.56137306e-01  9.57039865e-01
   1.17119421e+00 -1.23056533e+00  1.81016119e-01  5.23565423e-01]
 [ 8.31014697e-01  8.22981509e-04  1.52106971e-01 -4.96104150e-02
   2.78096315e-01  7.01630108e-01 -1.04268387e+00 -6.76145797e-01]]
V
 [[ 0.78027041  0.49315609 -0.96264874  1.37189624 -0.

#### Self attention

In [4]:
np.matmul(q,k.T)

array([[ 1.90236139,  0.17493851,  1.94411624, -3.17107159],
       [ 3.95125706, -1.11460719, -4.37591845,  2.69418582],
       [-3.70889645,  1.96140921,  1.91566919,  1.31749333],
       [ 1.46539482,  2.02522225,  1.4985709 ,  0.90399714]])

In [5]:
# now divide by sqrt(d_k) since we want to reduce the variance 
q.var(), k.var(), np.matmul(q,k.T).var() 

(1.0818524227722164, 0.5453739041230752, 5.472720986257096)

In [6]:
scaled = np.matmul(q,k.T)/math.sqrt(d_k)
q.var(), k.var(), scaled.var() 


(1.0818524227722164, 0.5453739041230752, 0.684090123282137)

In [7]:
# now we can see the reduction in variance of the producet 
scaled

array([[ 0.67258632,  0.0618501 ,  0.68734889, -1.12114311],
       [ 1.39698033, -0.39407315, -1.5471208 ,  0.95253853],
       [-1.31129291,  0.69346288,  0.67729134,  0.46580423],
       [ 0.51809531,  0.71602419,  0.52982482,  0.31961125]])

### Masking 
* This is to ensure words dont get context from words generated in future
* Not required in encoders(since all words are input at a time), but required in decoder 


In [8]:
mask = np.tril(np.ones((L,L)))
mask 

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [9]:
mask[mask==0] = -np.infty 
mask[mask==1] = 0

In [10]:
scaled + mask 

array([[ 0.67258632,        -inf,        -inf,        -inf],
       [ 1.39698033, -0.39407315,        -inf,        -inf],
       [-1.31129291,  0.69346288,  0.67729134,        -inf],
       [ 0.51809531,  0.71602419,  0.52982482,  0.31961125]])

In [11]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x),axis=-1)).T

In [12]:
attention = softmax(scaled + mask)

In [13]:
new_v = np.matmul(attention,v)
new_v

array([[ 0.78027041,  0.49315609, -0.96264874,  1.37189624, -0.04916138,
        -0.82434936,  0.20130451,  1.65966418],
       [ 0.66154134,  0.61021532, -0.83048732,  1.143522  ,  0.24448277,
        -0.85142504,  0.25850709,  1.49665143],
       [ 0.86424383,  0.80144813, -0.52735298, -0.26886184,  0.98070499,
        -0.35835217,  0.67995802,  1.52064345],
       [ 0.60494068,  0.41952798, -0.24694202,  0.05607446,  0.51001227,
        -0.57185013,  0.52195006,  1.32874961]])

In [14]:
v

array([[ 0.78027041,  0.49315609, -0.96264874,  1.37189624, -0.04916138,
        -0.82434936,  0.20130451,  1.65966418],
       [-0.05033032,  1.31207497, -0.03807837, -0.22575636,  2.00510429,
        -1.01376449,  0.60148031,  0.51926522],
       [ 1.80522334,  0.32469835, -0.96501699, -0.53727196,  0.08058237,
         0.37153528,  0.8252376 ,  2.51931674],
       [-0.11591258, -0.88000712,  1.20149022, -0.39754784, -0.50058359,
        -0.77109831,  0.42053471,  0.65937641]])

In [15]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x),axis=-1)).T

def scaled_dot_product_attention(q,k,v,mask=None):
    d_k = q.shape[-1]
    scaled = np.matmul(q,k.T)/math.sqrt(d_k)
    if mask is not None:
        scaled = scaled + mask 
    
    attention = softmax(scaled)
    out = np.matmul(attention,v)
    return out,attention 

In [16]:
 values, attention  = scaled_dot_product_attention(q,k,v,mask=mask)

In [17]:
print("Q\n",q)
print("K\n",k)
print("V\n",v)
print("New V \n", values )
print("Attention \n", attention)

Q
 [[-1.43357575 -0.43065121  0.77136426  0.19574309 -0.22401895 -1.04299148
   1.10825192  0.20315761]
 [-0.34115764 -1.5118362   0.01821113 -1.5611847  -0.76708133  2.09346769
  -0.67368897 -1.39134768]
 [ 0.53909654  0.6232494   1.74257407  1.2956597   0.18971572 -1.70160811
  -1.06915727 -1.02722077]
 [-0.74246299 -1.87555485  0.99599366  0.55042828 -0.56317419 -0.71420677
  -1.07514025 -1.38289993]]
K
 [[-1.36513690e+00 -3.63309409e-01 -5.99209478e-01 -9.82497463e-01
  -1.68603158e+00  4.11995122e-02  1.12483738e-01 -7.87179268e-02]
 [-1.16276551e+00  1.76990511e-01 -4.58214292e-01  1.15299911e-01
   6.41515596e-01 -7.75446731e-01 -1.57954247e+00  2.88702225e-03]
 [-1.02698577e-01 -9.33278768e-01 -1.56137306e-01  9.57039865e-01
   1.17119421e+00 -1.23056533e+00  1.81016119e-01  5.23565423e-01]
 [ 8.31014697e-01  8.22981509e-04  1.52106971e-01 -4.96104150e-02
   2.78096315e-01  7.01630108e-01 -1.04268387e+00 -6.76145797e-01]]
V
 [[ 0.78027041  0.49315609 -0.96264874  1.37189624 -0.

### Multi-Head Attention

In [18]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [19]:
sequence_length =4 
batch_size=1 
input_dim = 512 
d_model = 512 
x = torch.randn((batch_size,sequence_length,input_dim))

In [20]:
x.size()

torch.Size([1, 4, 512])

In [21]:
qkv_layer = nn.Linear(input_dim,3*d_model)

In [23]:
qkv = qkv_layer(x)

In [24]:
qkv.shape

torch.Size([1, 4, 1536])

In [26]:
num_heads = 8 
head_dim = d_model// num_heads
qkv = qkv.reshape(batch_size,sequence_length,num_heads,3*head_dim)

In [27]:
qkv.shape

torch.Size([1, 4, 8, 192])